In [0]:
import time
from selenium import webdriver
from itertools import count
from bs4 import BeautifulSoup
import pandas as pd


def question_head(bs):
    divs = bs.find("div",{"class":"civilqnaDiv"})
    ques_class = divs.find("div",{"class":"questionDiv"})
    
    if ques_class.find("th",{"class":"tit"})==None: 
        return "없음"
    else : 
        return ques_class.find("th",{"class":"tit"}).text.strip()
     

def question_body(bs):
    divs = bs.find("div",{"class":"civilqnaDiv"})
    ques_class = divs.find("div",{"class":"questionDiv"})
    
    if ques_class.find("td",{"colspan":"3"})==None:
        return "없음"
    else: 
        return ques_class.find("td",{"colspan":"3"}).text.strip()


def answer_body(bs):
    divs = bs.find("div",{"class":"civilqnaDiv"})
    answ_class = divs.find("div",{"class":"answerDiv"})
    answ_body = answ_class.find("div",{"class":"answerTxt"})
    
    if answ_class.find("div",{"class":"answerTxt"})==None: 
        return "없음"
    else : 
        return answ_class.find("div",{"class":"answerTxt"}).text.strip()
    

def department(bs):
    uls = bs.find("ul", {"class": "civilPart"})
    lis = uls.select('span[class = pl10]')
    check = lis[-1].text
    if check==None: 
        return "없음"
    else : 
        #lis = uls.find("li")
        return check.strip()
            
        
        
def law(bs):
    uls = bs.find("ul", {"class": "civilPart"})
    lis = uls.find("li",{"class":"mt5"})
    
    if uls.find("li",{"class":"mt5"})==None: 
        return "없음"
    else : 
        lis = uls.find("li",{"class":"mt5"})
        return lis.find("span").text.strip()

def other_deparment(bs):
    uls = bs.find("ul", {"class": "civilPart"})
    lis = uls.select('[class = mt5]')
    law_, department_ = lis[0].text.strip(), lis[-1].text.strip()
    return law_, department_
    
url = 'https://www.epeople.go.kr/jsp/user/pc/policy/uPcOpenCivilList.paid?iPage=&pageNo=&pageNum=64&strFlag=&strArea=&strBody=%C0%E5%BE%D6%C0%CE&cat_name=&strLaw=&strFrom_ex=2018-10-11&strTo_ex=2019-10-11&strFrom=20181011&strTo=20191011&show_sele=10&s_anc_c=&so_gubun1=&so_gubun2=&so_gubun3=&so_gubun4=&search_gubun1=&search_gubun2=&search_gubun3=&so_recom=&so_recom_Subanc='
    
browser = webdriver.Chrome("D:\chromedriver.exe")
browser.get(url)

result=pd.DataFrame()

for page in count(64):
    print("Current page : ",page)
    script = 'gotoPage(%d)'%page        ## page 넘기기
    browser.execute_script(script)
    
        
    html_final_check = browser.page_source   ## 마지막 page 체크
    bs_final = BeautifulSoup(html_final_check,'html.parser')
    if bs_final.find("td",{"class":"b_text"}) != None:  
        break  

    
    web_list = browser.find_elements_by_css_selector('td > a')   ##해당 페이지에 있는 글 정보 추출
    web_urls = [ i.get_attribute('href') for i in web_list ]
    
    for i in web_urls:
        browser.execute_script(i)
        time.sleep(3)
        
        html = browser.page_source         ## 글 정보 추출하기
        bs = BeautifulSoup(html, 'html.parser')
        
        try:
            ques_head = question_head(bs)
            ques_body = question_body(bs)
            answ_body = answer_body(bs)
            dep_body = department(bs)
            law_body = law(bs)
            
            df1=pd.DataFrame({ 'title' : [str(ques_head)], 
                    'content' : [str(ques_body)],
                   'answer' :  [str(answ_body)], 
                    'department' : [str(dep_body)], 
                    'law': [str(law_body)]
            })

            #print("질문제목:",ques_head,"\n","질문내용:",ques_body,"\n","답변:" ,answ_body,"\n","담당부서",dep_body,"\n","관련법령",law_body,"\n")
            result=pd.concat([result, df1])
            browser.find_element_by_xpath("//a[@onclick='goList();']").click()  ## 목록으로 되돌아가기
            time.sleep(3)
            
        except IndexError :
            ques_head = question_head(bs)
            ques_body = question_body(bs)
            answ_body = answer_body(bs)
            law_body, dep_body = other_deparment(bs)
            
            df1=pd.DataFrame({ 'title' : [str(ques_head)], 
                    'content' : [str(ques_body)],
                   'answer' :  [str(answ_body)], 
                    'department' : [str(dep_body)], 
                    'law': [str(law_body)]
            })

            #print("질문제목:",ques_head,"\n","질문내용:",ques_body,"\n","답변:" ,answ_body,"\n","담당부서",dep_body,"\n","관련법령",law_body,"\n")
            result=pd.concat([result, df1])
            browser.find_element_by_xpath("//a[@onclick='goList();']").click()  ## 목록으로 되돌아가기
            time.sleep(3)

Current page :  64


WebDriverException: Message: chrome not reachable
  (Session info: chrome=77.0.3865.90)


In [0]:
#result.index = np.arange(len(result))
#result.to_pickle("1004.pkl")
result1 = pd.read_pickle("632.pkl")
result2 = pd.read_pickle("1004.pkl")
result = pd.concat([result1, result2],axis = 0)
result.drop_duplicates(inplace = True)
result.index = np.arange(len(result))
result.to_excel("epeople_data.xlsx", index = False)